# Purpose

This notebook is for sanity checks of the our implemented contextual decomposition of GPTs.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import torch
import warnings
import random
import itertools

warnings.filterwarnings("ignore")
base_dir = os.path.split(os.getcwd())[0]
sys.path.append(base_dir)

from pyfunctions.cdt_basic import *
from pyfunctions.cdt_source_to_target import *
from pyfunctions.cdt_from_source_nodes import *
from pyfunctions.ioi_dataset import IOIDataset
from pyfunctions.wrappers import *
from transformers import AutoTokenizer, AutoModel
from transformers import GPT2Tokenizer, GPT2Model
from transformer_lens import utils, HookedTransformer, ActivationCache
from fancy_einsum import einsum


In [3]:
torch.autograd.set_grad_enabled(False)

In [4]:
device = 'cuda:0'

In [ ]:
# Model code adapted from Callum McDougall's notebook for ARENA on reproducing the IOI paper using TransformerLens.
model = HookedTransformer.from_pretrained("gpt2-small",
                                          center_unembed=True,
                                          center_writing_weights=True,
                                          fold_ln=False,
                                          refactor_factored_attn_matrices=True)                                         

In [6]:
model.cfg.default_prepend_bos = False

## Model inspection

In [ ]:
model.cfg.default_prepend_bos # prepends a beginning-of-sequence token?
model.tokenizer.padding_side # 'right'
model.cfg.positional_embedding_type # 'standard', as opposed to 'shortformer', which puts the positional embedding in the attention pattern
model.cfg

## Inspect model / hooks


In [8]:
text = "After John and Mary went to the store, John gave a bottle of milk to"
tokens = model.to_tokens(text).to(device)
logits, cache = model.run_with_cache(tokens)
for activation_name, activation in cache.items():
    # Only print for first layer
    if ".0." in activation_name or "blocks" not in activation_name:
        print(f"{activation_name:30} {tuple(activation.shape)}")

hook_embed                     (1, 16, 768)
hook_pos_embed                 (1, 16, 768)
blocks.0.hook_resid_pre        (1, 16, 768)
blocks.0.ln1.hook_scale        (1, 16, 1)
blocks.0.ln1.hook_normalized   (1, 16, 768)
blocks.0.attn.hook_q           (1, 16, 12, 64)
blocks.0.attn.hook_k           (1, 16, 12, 64)
blocks.0.attn.hook_v           (1, 16, 12, 64)
blocks.0.attn.hook_attn_scores (1, 12, 16, 16)
blocks.0.attn.hook_pattern     (1, 12, 16, 16)
blocks.0.attn.hook_z           (1, 16, 12, 64)
blocks.0.hook_attn_out         (1, 16, 768)
blocks.0.hook_resid_mid        (1, 16, 768)
blocks.0.ln2.hook_scale        (1, 16, 1)
blocks.0.ln2.hook_normalized   (1, 16, 768)
blocks.0.mlp.hook_pre          (1, 16, 3072)
blocks.0.mlp.hook_post         (1, 16, 3072)
blocks.0.hook_mlp_out          (1, 16, 768)
blocks.0.hook_resid_post       (1, 16, 768)
ln_final.hook_scale            (1, 16, 1)
ln_final.hook_normalized       (1, 16, 768)


## Test correctness of forward pass

In [9]:
source_list = [(Node(0, 0, 0),), (Node(1, 1, 1),)]
target_nodes = []


encoding = model.tokenizer.encode_plus(text, 
                                 add_special_tokens=True, 
                                 max_length=512,
                                 truncation=True, 
                                 padding = "longest", 
                                 return_attention_mask=True, 
                                 return_tensors="pt").to(device)
encoding_idxs, attention_mask = encoding.input_ids, encoding.attention_mask
input_shape = encoding_idxs.size()
extended_attention_mask = get_extended_attention_mask(attention_mask, 
                                                        input_shape, 
                                                        model,
                                                        device)
out_decomps, target_decomps, _, _ = prop_GPT(encoding_idxs, extended_attention_mask, model, source_list, target_nodes, device=device, mean_acts=None, set_irrel_to_mean=False)

In [10]:
# This loosely verifies that the out-decomposition's rel + irrel components equals the output of the forward pass
compare_same(out_decomps[0].rel + out_decomps[0].irrel, out_decomps[1].rel + out_decomps[1].irrel)

# compare_same(cache['blocks.0.attn.hook_q'], cache['blocks.0.attn.hook_k']) # checking that there aren't the inputs, because the source code names them as such

99.64% of the values are equal


0.9964407943172096

In [24]:
print(embedding_output.shape)

torch.Size([1, 16, 768])


In [11]:
# compare_same(cache['hook_embed'], cache['blocks.0.hook_resid_pre'])
# This just verifies an assumption about the input conventions to the model.
in_0 = cache['blocks.0.hook_resid_pre']
compare_same(cache['hook_embed'] + cache['hook_pos_embed'], cache['blocks.0.hook_resid_pre'])

# Ensure that the encodings are the same (they won't be if you set prepend_bos=True for one and not the other!)
# print(tokens)
# print(encoding.input_ids)

# Note that model.embed alone isn't the same thing as what we might normally call "the embedding"!
embedding_output = model.embed(encoding.input_ids) + model.pos_embed(encoding.input_ids)
compare_same(in_0, embedding_output)



100.00% of the values are equal
100.00% of the values are equal


1.0

In [ ]:
# Extra analysis of value matrix calcuation
print(model.blocks[0].attn.W_V.shape)
print(model.blocks[0].attn.W_O.shape)
# print(model.blocks[0].attn.hook_pattern) # 1, 12, seq_len, seq_len



In [ ]:
# Test equivalence of attention layer
'''
blocks.0.attn.hook_q           (1, 17, 12, 64)
blocks.0.attn.hook_k           (1, 17, 12, 64)
blocks.0.attn.hook_v           (1, 17, 12, 64)
blocks.0.attn.hook_attn_scores (1, 12, 17, 17)
blocks.0.attn.hook_pattern     (1, 12, 17, 17)
blocks.0.attn.hook_z           (1, 17, 12, 64)
'''

input_shape = encoding['input_ids'].size()

attention_mask = get_extended_attention_mask(encoding['attention_mask'], 
                                                        input_shape, 
                                                        model,
                                                        device)
sh = list(embedding_output.shape)
rel = torch.zeros(sh, dtype = embedding_output.dtype, device = device)
irrel = torch.zeros(sh, dtype = embedding_output.dtype, device = device)
irrel[:] = embedding_output[:]
layer_module = model.blocks[0]
attention_module = layer_module.attn


rel_ln, irrel_ln = prop_layer_norm(rel, irrel, GPTLayerNormWrapper(layer_module.ln1))

In [42]:
attn_probs = prop_attention_probs(rel_ln, irrel_ln, attention_mask, 
                                                                        None,
                                                                        attn_wrapper)
desired_attn_probs = cache['blocks.0.attn.hook_pattern']
'''
comparison = (np.isclose(desired_scores[0, 0, -2, :-1].cpu().numpy(), scores[0, 0, -2, :-1].cpu().numpy(), atol=1e-2, rtol=1e-2))
print(comparison)
proportion = comparison.sum() / comparison.size
print(proportion)
'''
# print(desired_scores.shape)
# print(desired_attn_probs[0, 0, -2, :])
#print(scores[0, 0, -2, :])

compare_same(attn_probs, desired_attn_probs, atol=1e-2, rtol=1e-2)

tensor([0.0985, 0.1019, 0.0676, 0.1372, 0.0322, 0.0466, 0.0637, 0.0243, 0.0416,
        0.0757, 0.0220, 0.0363, 0.0937, 0.0648, 0.0938, 0.0000],
       device='cuda:0')
100.00% of the values are equal


1.0

In [44]:

'''
print("LayerNorm: ")
compare_same(cache['blocks.0.ln1.hook_normalized'], (rel_ln + irrel_ln)[0, :16, :])

desired_value_output = cache['blocks.0.attn.hook_v']
rel_value, irrel_value = prop_linear(rel_ln, irrel_ln, GPTAttentionWrapper(attention_module, layer_module.ln2).value)
value = rel_value + irrel_value
print("Value matrix:")
# desired_value_output = desired_value_output.transpose(-1, -2)
old_shape = desired_value_output.size()
new_shape = old_shape[:-2] + (old_shape[-2] * old_shape[-1],)
desired_value_output = desired_value_output.reshape(new_shape)
print(desired_value_output.shape)
print(value.shape)

compare_same(value[0, :16, :], desired_value_output, rtol = 1e-2) # some elements differ by more than 1e-3 in proportion, not great
'''

# desired_block_output = cache['blocks.0.hook_resid_mid']

desired_block_output = cache['blocks.0.hook_attn_out']
desired_attention_pattern = cache['blocks.0.attn.hook_pattern']
# level = 0
# source_node_list = [(0, 0, 0), (1, 1, 1)]
# target_nodes = [(7, 82, 11), (7, 82, 0), (7, 82, 6), (9, 82, 0), (9, 91, 7), (8, 82, 0)]
attn_wrapper = GPTAttentionWrapper(attention_module)

rel_summed_values, irrel_summed_values, returned_att_probs = prop_attention_no_output_hh(rel_ln, irrel_ln, attention_mask, 
                                                                        None,
                                                                        attn_wrapper,
                                                                        att_probs=None)
print("'Z' matrix:")
desired_z = cache['blocks.0.attn.hook_z']
old_shape = desired_z.size()
new_shape = old_shape[:-2] + (old_shape[-2] * old_shape[-1],)
desired_z = desired_z.reshape(new_shape)
z = rel_summed_values + irrel_summed_values
compare_same(desired_z, z[:, :16, :])

rel_attn_residual, irrel_attn_residual = prop_linear(rel_summed_values, irrel_summed_values, attn_wrapper.output)

print("Output matrix:")
output = rel_attn_residual + irrel_attn_residual
# output = rel_attn_residual + irrel_attn_residual + rel_ln + irrel_ln
compare_same(output[:, :16, :], desired_block_output)


desired_mid_output = cache['blocks.0.hook_resid_mid']
rel_mid, irrel_mid = rel + rel_attn_residual, irrel + irrel_attn_residual
mid_output = rel_mid + irrel_mid
compare_same(mid_output[:, :16, :], desired_mid_output)




'Z' matrix:
99.71% of the values are equal
Output matrix:
99.89% of the values are equal
99.89% of the values are equal


0.9989420572916666

In [48]:
# Test equivalence of entire transformer block
in_1 = cache['blocks.1.hook_resid_pre']
input_shape = encoding['input_ids'].size()

extended_attention_mask = get_extended_attention_mask(encoding['attention_mask'], 
                                                        input_shape, 
                                                        model,
                                                        device)
sh = list(embedding_output.shape)
rel = torch.zeros(sh, dtype = embedding_output.dtype, device = device)
irrel = torch.zeros(sh, dtype = embedding_output.dtype, device = device)
irrel[:] = embedding_output[:]
layer_module = model.blocks[0]

ablation_dict = {}
target_nodes = [] # [(7, 82, 11), (7, 82, 0), (7, 82, 6), (9, 82, 0), (9, 91, 7), (8, 82, 0)]
rel_out, irrel_out, layer_target_decomps, _ = prop_GPT_layer(rel, irrel, extended_attention_mask, 
                                                                                 None, ablation_dict, 
                                                                                 target_nodes, 0, 
                                                                                 None,
                                                                                 layer_module, 
                                                                                 device,
                                                                                 None)

layer0_output = rel_out + irrel_out
compare_same(in_1, layer0_output[:, :16, :])

# want to check: layer_target_decomps is what we think it is (currently gated by, i didn't implement the decomposition part)
# also, rel + irrel sums to the correct output
# layer_target_decomps[0].rel + layer_target_decomps[0].irrel == layer_target_decomps[1].rel + layer_target_decomps[1].irrel


99.97% of the values are equal


0.9996744791666666

In [50]:
# Test equivalence of model run to logits
print(logits.shape)
logits, cache = model.run_with_cache(tokens)
out_decomps, target_decomps, _, _ = prop_GPT(encoding_idxs, extended_attention_mask, model, source_list, target_nodes, device=device, mean_acts=None, set_irrel_to_mean=False)

model_out = out_decomps[0].rel + out_decomps[0].irrel
print(model_out.shape)
compare_same(logits[0], model_out)

torch.Size([1, 16, 50257])
(1, 16, 50257)
100.00% of the values are equal


1.0

## Test correctness of batching

In [10]:
# Generate a dataset all consisting of one template, randomly chosen.
# nb_templates = 2 due to some logic internal to IOIDataset:
# essentially, the nouns can be an ABBA or ABAB order and that counts as separate templates.
ioi_dataset = IOIDataset(prompt_type="mixed", N=3, tokenizer=model.tokenizer, prepend_bos=False, nb_templates=2)

# This is the P_ABC that is mentioned in the IOI paper, which we use for mean ablation.
# Importantly, passing in prompt_type="ABC" or similar is NOT the same thing as this.
abc_dataset = (
    ioi_dataset.gen_flipped_prompts(("IO", "RAND"))
    .gen_flipped_prompts(("S", "RAND"))
    .gen_flipped_prompts(("S1", "RAND"))
)

2024-08-30 13:20:47.277592: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-30 13:20:51.596009: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [11]:
logits, cache = model.run_with_cache(abc_dataset.toks) # run on entire dataset along batch dimension
attention_outputs = [cache['blocks.' + str(i) + '.hook_attn_out'] for i in range(12)]
attention_outputs = torch.stack(attention_outputs, dim=1) # now batch, head, seq, d_model
mean_acts = torch.mean(attention_outputs, dim=0)
mean_acts.shape


torch.Size([12, 16, 768])

In [24]:
text = ioi_dataset.sentences[0]
encoding = model.tokenizer.encode_plus(text, 
                                 add_special_tokens=True, 
                                 max_length=512,
                                 truncation=True, 
                                 padding = "longest", 
                                 return_attention_mask=True, 
                                 return_tensors="pt").to(device)
encoding_idxs, attention_mask = encoding.input_ids, encoding.attention_mask
input_shape = encoding_idxs.size()
extended_attention_mask = get_extended_attention_mask(attention_mask, 
                                                        input_shape, 
                                                        model,
                                                        device)

import functools
ranges = [
        [layer for layer in range(11, 12)],
        [sequence_position for sequence_position in range(input_shape[-1])],
        [attention_head_idx for attention_head_idx in range(1)]
    ]

source_nodes = list(itertools.product(*ranges))
# print(source_nodes[:64])
# target_nodes = [(7, 0, 1)]
target_nodes = []

prop_fn = lambda snl: prop_GPT(encoding_idxs[0:1, :], extended_attention_mask, model, snl, target_nodes=target_nodes, device=device, mean_acts=mean_acts, set_irrel_to_mean=True)
batch_out_decomps, target_decomps = batch_run(prop_fn, source_nodes)
iter_out_decomps = []

# run one by one
for layer in range(11, 12):
    for sequence_position in range(input_shape[-1]):
        for attention_head_idx in range(1):
            source_node = (layer, sequence_position, attention_head_idx)
            target_nodes = []
            out_decomp, _, _ = prop_GPT(encoding_idxs[0:1, :], extended_attention_mask, model, [source_node], target_nodes, mean_acts=mean_acts, set_irrel_to_mean=True, device=device)
            iter_out_decomps.append(out_decomp[0])
            


In [ ]:
# run in one batch
batch_out_decomps, _, _ = prop_GPT(encoding_idxs[0:1, :], extended_attention_mask, model, source_nodes, target_nodes=target_nodes, device=device, mean_acts=mean_acts, set_irrel_to_mean=True)

In [ ]:
print(batch_out_decomps[0].rel[0][4][11])
print(iter_out_decomps[0].rel[0][4][11])
print(batch_out_decomps[0].source_node)

In [ ]:
for seq_pos in range(16):
    for head_idx in range(12):
        print(seq_pos, head_idx)
        compare_same(batch_out_decomps[0].rel[0][seq_pos][head_idx], iter_out_decomps[0].rel[0][seq_pos][head_idx])


In [25]:
compare_same(batch_out_decomps[0].rel, iter_out_decomps[0].rel)
# compare_same(manual_batch_out_decomps[0].rel, iter_out_decomps[0].rel)
# compare_same(manual_batch_out_decomps[0].rel, batch_out_decomps[0].rel)



100.00% of the values are correct


In [ ]:
for x in range(16):
    batch = batch_out_decomps[x]
    it = iter_out_decomps[x]
    print(batch.rel.shape, it.rel.shape)